In [ ]:
# 필요한 라이브러리 임포트
import os
import sys
import json
import numpy as np
import cv2
from PIL import Image
from pathlib import Path
from tqdm import tqdm

try:
    import lxml.etree as etree
except ImportError:
    import xml.etree.ElementTree as etree
    print("경고: lxml 라이브러리를 찾을 수 없어 표준 xml.etree.ElementTree를 사용합니다.")

# --- Matterport Mask R-CNN 의존성 추가 ---
# 이 스크립트를 실행하려면 Matterport Mask_RCNN 라이브러리가 필요합니다.
# 위 '환경 설정' 가이드를 따라 'mrcnn' 폴더를 준비해주세요.
# ---------------------------------------------
try:
    from mrcnn.config import Config
    from mrcnn import model as modellib
except ImportError:
    print("-" * 80)
    print("오류: Matterport Mask R-CNN 라이브러리를 찾을 수 없습니다.")
    print("실행 방법:")
    print("1. 'git clone https://github.com/matterport/Mask_RCNN.git' 명령어로 저장소를 클론하세요.")
    print("2. 클론된 'Mask_RCNN' 폴더 안에 있는 'mrcnn' 폴더를 이 스크립트가 있는 디렉토리로 복사하세요.")
    print("-" * 80)
    sys.exit(1)


class OrganDatasetParser:
    """장기 데이터셋 폴더 구조를 동적으로 파싱하는 클래스 (프레임워크 비종속적이므로 변경 없음)"""
    def __init__(self, base_path, image_dir_name="원천데이터", label_dir_name="라벨링데이터"):
        self.base_path = Path(base_path)
        self.label_path = self.base_path / label_dir_name
        self.image_path = self.base_path / image_dir_name
        self.organ_categories = self._discover_organ_categories()

        if not self.organ_categories:
            raise ValueError(f"'{self.label_path}' 경로에서 장기 클래스 폴더를 찾을 수 없습니다. 폴더 구조를 확인해주세요.")

        self.class_to_idx = {cat['name']: cat['id'] for cat in self.organ_categories}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        print(f"자동으로 발견된 장기 카테고리: {len(self.organ_categories)}개")
        for cat in self.organ_categories:
            print(f"  - ID {cat['id']}: {cat['name']}")

    def _discover_organ_categories(self):
        """라벨링데이터 폴더의 하위 폴더 이름을 클래스로 인식하여 카테고리 자동 생성"""
        categories = []
        if self.label_path.exists() and self.label_path.is_dir():
            organ_folders = sorted([f for f in self.label_path.iterdir() if f.is_dir()])
            # Matterport Mask R-CNN은 클래스 ID 0을 배경(BG)으로 예약하므로 실제 클래스는 1부터 시작합니다.
            categories = [{"id": idx + 1, "name": folder.name, "supercategory": "organ"} for idx, folder in enumerate(organ_folders)]
        return categories


class OrganInferenceConfig(Config):
    """Matterport Mask R-CNN을 위한 추론(Inference) 설정"""
    def __init__(self, num_classes, gpu_count=1, images_per_gpu=1):
        self.NAME = "organ_inference"
        self.GPU_COUNT = gpu_count
        self.IMAGES_PER_GPU = images_per_gpu
        # 클래스 수 + 1 (배경 클래스 BG 포함)
        self.NUM_CLASSES = 1 + num_classes
        super().__init__()
    
    # 추론 성능을 위해 일부 파라미터 고정
    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 1024
    DETECTION_MIN_CONFIDENCE = 0.7


class OrganDetector:
    """학습된 Keras/TF1.x 모델로 장기를 탐지하는 추론 클래스"""
    def __init__(self, model_path, parser_obj: OrganDatasetParser, confidence_threshold=0.7):
        print("TensorFlow/Keras 기반 장기 탐지 모델을 초기화합니다...")
        num_classes = len(parser_obj.organ_categories)
        self.config = OrganInferenceConfig(num_classes)
        self.config.DETECTION_MIN_CONFIDENCE = confidence_threshold
        
        # 모델 디렉토리 지정 (로그 파일 저장 위치)
        model_dir = str(Path(model_path).parent)

        # 추론 모드로 모델 생성
        self.model = modellib.MaskRCNN(mode="inference", config=self.config, model_dir=model_dir)

        # 가중치 로드
        if not Path(model_path).exists():
            raise FileNotFoundError(f"모델 가중치 파일(.h5) '{model_path}'를 찾을 수 없습니다.")
        print(f"가중치 로딩: {model_path}")
        self.model.load_weights(model_path, by_name=True)
        print("모델 로딩 완료.")

    def detect(self, image_path):
        """이미지 경로를 입력받아 장기를 탐지하고 결과를 반환"""
        image_np = np.array(Image.open(image_path).convert('RGB'))
        
        # 모델 추론 실행
        results = self.model.detect([image_np], verbose=0)
        r = results[0] # 첫 번째 이미지에 대한 결과

        # Matterport 결과: 'rois', 'masks', 'class_ids', 'scores' (모두 NumPy 배열)
        return Image.fromarray(image_np), r


class CameraMotionEstimator:
    """두 이미지 간 카메라 움직임 추정 클래스 (OpenCV 기반 로직)"""
    def __init__(self, organ_detector: OrganDetector):
        self.detector = organ_detector
        self.lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    def _extract_features_from_organs(self, image_cv, detection_result):
        """탐지된 장기 영역 내에서만 특징점을 추출 (Matterport 출력 형식에 맞게 수정)"""
        masks = detection_result['masks'] # Shape: [H, W, N]
        if masks.shape[-1] == 0:
            return None, "No masks detected."

        # 모든 장기 마스크(N개)를 하나의 마스크로 합침
        combined_mask = np.any(masks, axis=-1).astype(np.uint8)

        if np.sum(combined_mask) > 0:
            features = cv2.goodFeaturesToTrack(cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY),
                                               maxCorners=100, qualityLevel=0.3, minDistance=7, mask=combined_mask)
            return features, "Success"
        return None, "Combined mask is empty."

    def estimate_motion(self, image1_path: str, image2_path: str):
        # 1. 두 이미지에서 각각 장기 탐지
        pil_img1, detections1 = self.detector.detect(image1_path)
        pil_img2, detections2 = self.detector.detect(image2_path)
        
        cv_img1 = cv2.cvtColor(np.array(pil_img1), cv2.COLOR_RGB2BGR)
        cv_img2 = cv2.cvtColor(np.array(pil_img2), cv2.COLOR_RGB2BGR)

        # 2. 첫 번째 이미지의 장기 영역에서 특징점 추출
        p0, status_msg = self._extract_features_from_organs(cv_img1, detections1)
        if p0 is None:
            return None, f"No features to track in first image: {status_msg}"
            
        # 3. 광학 흐름(Optical Flow)으로 특징점 추적
        p1, status, _ = cv2.calcOpticalFlowPyrLK(cv2.cvtColor(cv_img1, cv2.COLOR_BGR2GRAY), 
                                                 cv2.cvtColor(cv_img2, cv2.COLOR_BGR2GRAY), 
                                                 p0, None, **self.lk_params)
        
        if p1 is None or status is None or len(p1[status == 1]) < 5:
             return None, "Optical flow tracking failed or not enough points tracked."

        good_new = p1[status == 1]
        good_old = p0[status == 1]
        
        # 4. 카메라 움직임 계산
        # 좌우/상하 이동 (Translation)
        dx, dy = np.mean(good_new - good_old, axis=0)
        
        # 앞뒤 이동 (Zoom) - BBox 크기 변화로 추정
        boxes1 = detections1['rois'] # Format: (y1, x1, y2, x2)
        boxes2 = detections2['rois']
        
        if len(boxes1) > 0 and len(boxes2) > 0:
            area1 = np.mean([(b[2] - b[0]) * (b[3] - b[1]) for b in boxes1])
            area2 = np.mean([(b[2] - b[0]) * (b[3] - b[1]) for b in boxes2])
            zoom_ratio = np.sqrt(area2 / area1) if area1 > 0 else 1.0
        else:
            zoom_ratio = 1.0 # BBox 탐지 실패 시 줌 비율은 1로 고정
            
        return {"translation_x": dx, "translation_y": dy, "zoom_ratio": zoom_ratio}, "Success"


if __name__ == '__main__':
    # !!! 사용자 수정 필요: 경로 및 설정 !!!
    PROJECT_ROOT = Path

    # 1. 데이터셋 전체의 기본 경로
    DATASET_BASE_PATH = PROJECT_ROOT / "surgical_dataset"
    
    # 2. 학습된 Matterport Mask R-CNN 모델 가중치 파일(.h5) 경로
    #    PyTorch 모델(.pth)은 호환되지 않습니다.
    MODEL_WEIGHTS_H5_PATH = PROJECT_ROOT / "trained_models" / "mask_rcnn_organ.h5"
    
    # 3. 분석할 이미지 시퀀스가 있는 폴더 이름 (예: 'liver_sequence_01')
    TARGET_SEQUENCE_FOLDER = "liver_sequence_01"
    
    # 4. 결과 JSON 파일을 저장할 경로
    OUTPUT_JSON_PATH = PROJECT_ROOT / f"motion_analysis_{TARGET_SEQUENCE_FOLDER}_tf.json"
    
    # ----------------------------------------
    
    try:
        # --- 1. 초기화 단계 ---
        print("스크립트 실행을 시작합니다. Keras/TensorFlow 1.13 환경을 사용합니다.")
        
        organ_parser = OrganDatasetParser(DATASET_BASE_PATH)
        detector = OrganDetector(str(MODEL_WEIGHTS_H5_PATH), organ_parser, confidence_threshold=0.7)
        motion_estimator = CameraMotionEstimator(detector)

        # --- 2. 이미지 시퀀스 처리 단계 ---
        sequence_image_path = DATASET_BASE_PATH / "원천데이터" / TARGET_SEQUENCE_FOLDER
        if not sequence_image_path.exists():
            raise FileNotFoundError(f"이미지 시퀀스 폴더를 찾을 수 없습니다: {sequence_image_path}")
        
        image_files = sorted(sequence_image_path.glob('*.[jp][pn]g')) # .jpg, .jpeg, .png
        if len(image_files) < 2:
            print(f"'{sequence_image_path}' 폴더에 비교할 이미지가 2장 이상 없습니다.")
            sys.exit(0)
            
        print(f"\n총 {len(image_files)}개의 이미지에서 {len(image_files)-1}개의 프레임 쌍을 분석합니다.")
        
        all_motion_data = []

        # 순차적으로 프레임 쌍 비교
        for i in tqdm(range(len(image_files) - 1), desc=f"Processing '{TARGET_SEQUENCE_FOLDER}'"):
            image1_path = image_files[i]
            image2_path = image_files[i+1]
            
            motion_results, status_msg = motion_estimator.estimate_motion(str(image1_path), str(image2_path))
            
            frame_data = {
                "frame_start": image1_path.name,
                "frame_end": image2_path.name,
            }
            if motion_results:
                frame_data["motion_estimation"] = {
                    "translation_x_px": motion_results['translation_x'],
                    "translation_y_px": motion_results['translation_y'],
                    "zoom_ratio": motion_results['zoom_ratio']
                }
                frame_data["status"] = "Success"
            else:
                frame_data["motion_estimation"] = None
                frame_data["status"] = f"Failed: {status_msg}"
                print(f"\n경고: {image1_path.name} -> {image2_path.name} 움직임 추정 실패. 원인: {status_msg}")

            all_motion_data.append(frame_data)

        # --- 3. 결과 저장 단계 ---
        if all_motion_data:
            print(f"\n분석 완료. 결과를 '{OUTPUT_JSON_PATH}' 파일에 저장합니다.")
            with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump(all_motion_data, f, ensure_ascii=False, indent=4)
            print("저장 완료.")
        else:
            print("분석된 데이터가 없어 파일을 저장하지 않습니다.")

    except (FileNotFoundError, ValueError) as e:
        print(f"\n[오류] 스크립트 실행 중단: {e}")
        print("스크립트 상단의 DATASET_BASE_PATH, MODEL_WEIGHTS_H5_PATH, TARGET_SEQUENCE_FOLDER 경로 및 이름이 올바른지 확인해주세요.")
    except Exception as e:
        import traceback
        print(f"\n[예상치 못한 오류] 스크립트 실행 중단: {e}")
        traceback.print_exc()

